In [232]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

In [234]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

In [235]:
examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

In [236]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "영화 {movie}와 잘 어울리는 이모티콘을 추천 해줄래?"),
        ("ai", "{answer}"),
    ]
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [237]:
memory_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 내용과 잘 어울리는 이모티콘을 추천해 주는 챗봇이다. 질문으로 받은 영화에 어울리는 이모티몬을 제공하세요. 답변할 때 샘플로 제공한 영화는 제외하세요.",
        ),
        fewshot_prompt,
        MessagesPlaceholder(
            variable_name="history",
        ),
        ("human", "{movie}"),
    ]
)

In [238]:
memory_chain = RunnablePassthrough.assign(history=load_memory) | memory_prompt | llm

In [239]:
def invoke_chain(movie):
    result = memory_chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    # print(result)

In [241]:
invoke_chain("소방관")

👨‍🚒🔥🚒

In [242]:
invoke_chain("스턴트맨")

🎬🚗💥

In [248]:
load_memory({})

[HumanMessage(content='소방관', additional_kwargs={}, response_metadata={}),
 AIMessage(content='👨\u200d🚒🔥🚒', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='스턴트맨', additional_kwargs={}, response_metadata={}),
 AIMessage(content='🎬🚗💥', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what wat the movie name i asked you first?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm sorry, I cannot provide the movie name you asked for first. How can I assist you further?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='소방관을 물어본적있어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 소방관에 대한 이모티콘을 이미 제공했습니다. 다른 질문이 있으시면 도와드리겠습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내가 맨 처음 질문한 영화 제목은?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='죄송합니다. 제가 사용한 샘플 영화는 "Top Gun"이었습니다. 다른 질문이 있으시면 도와드리겠습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessag

In [247]:
invoke_chain("내가 이전에 질문한 영화 제목은?")

죄송합니다. 이전에 질문하신 영화 제목은 "The Godfather"였습니다. 다른 질문이 있으시면 도와드리겠습니다.

In [246]:
invoke_chain("내가 맨 처음 질문한 영화 제목은?")

죄송합니다. 제가 사용한 샘플 영화는 "Top Gun"이었습니다. 다른 질문이 있으시면 도와드리겠습니다.

In [245]:
invoke_chain("소방관을 물어본적있어?")

네, 소방관에 대한 이모티콘을 이미 제공했습니다. 다른 질문이 있으시면 도와드리겠습니다.

In [244]:
invoke_chain("what wat the movie name i asked you first?")

I'm sorry, I cannot provide the movie name you asked for first. How can I assist you further?